In [1]:
import os
import warnings
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configuration
RANDOM_STATE = 42
SAMPLE_SIZE = 200_000
TARGET_COLUMN = 'Label'

# Paths
PROJECT_ROOT = Path('..').resolve()
DATA_PATH = PROJECT_ROOT / 'data' / 'processed_randomforest' / 'train.csv'
RESULTS_DIR = PROJECT_ROOT / 'experiments' / 'results'
MODELS_DIR = PROJECT_ROOT / 'models'

# Ensure output directories exist
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Data Path: {DATA_PATH}")
print(f"📊 Results Dir: {RESULTS_DIR}")
print(f"🤖 Models Dir: {MODELS_DIR}")
print(f"✅ Configuration loaded!")

📂 Data Path: D:\Projects\networkdetection\networkdetection\data\processed_randomforest\train.csv
📊 Results Dir: D:\Projects\networkdetection\networkdetection\experiments\results
🤖 Models Dir: D:\Projects\networkdetection\networkdetection\models
✅ Configuration loaded!


In [2]:
# Load the pre-processed dataset
print("⏳ Loading dataset (this may take a moment for large files)...")
df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset loaded! Shape: {df.shape}")
print(f"   Total samples: {df.shape[0]:,}")
print(f"   Features: {df.shape[1] - 1}")
print(f"   Target: '{TARGET_COLUMN}'")

⏳ Loading dataset (this may take a moment for large files)...
✅ Dataset loaded! Shape: (1306484, 21)
   Total samples: 1,306,484
   Features: 20
   Target: 'Label'


In [3]:
# Display original class distribution
print("\n📊 Original Class Distribution:")
print("=" * 50)
class_dist = df[TARGET_COLUMN].value_counts().sort_index()
class_pct = df[TARGET_COLUMN].value_counts(normalize=True).sort_index() * 100

for label, count in class_dist.items():
    pct = class_pct[label]
    print(f"  Class {label}: {count:>10,} samples ({pct:>6.2f}%)")

print(f"\n  Total: {df.shape[0]:,} samples")


📊 Original Class Distribution:
  Class 0:  1,078,393 samples ( 82.54%)
  Class 1:    228,091 samples ( 17.46%)

  Total: 1,306,484 samples


In [4]:
# Strategic Sampling with Stratification
print(f"\n⚡ Creating stratified sample of {SAMPLE_SIZE:,} rows...")

# Separate features and target
X = df.drop(columns=[TARGET_COLUMN])
y = df[TARGET_COLUMN]

# Calculate sample fraction
sample_frac = SAMPLE_SIZE / len(df)

# Stratified sampling using train_test_split
# We take SAMPLE_SIZE as our "train" portion
X_sample, _, y_sample, _ = train_test_split(
    X, y,
    train_size=SAMPLE_SIZE,
    stratify=y,
    random_state=RANDOM_STATE
)

# Combine back into single DataFrame for PyCaret
df_sample = pd.concat([X_sample, y_sample], axis=1).reset_index(drop=True)

print(f"✅ Stratified sample created! Shape: {df_sample.shape}")


⚡ Creating stratified sample of 200,000 rows...
✅ Stratified sample created! Shape: (200000, 21)


In [5]:
# Verify stratification integrity
print("\n🔍 Sample Class Distribution (Integrity Check):")
print("=" * 60)

sample_dist = df_sample[TARGET_COLUMN].value_counts().sort_index()
sample_pct = df_sample[TARGET_COLUMN].value_counts(normalize=True).sort_index() * 100
original_pct = class_pct

print(f"{'Class':<10} {'Count':>12} {'Sample %':>12} {'Original %':>12} {'Δ':>8}")
print("-" * 60)

for label in sample_dist.index:
    count = sample_dist[label]
    s_pct = sample_pct[label]
    o_pct = original_pct[label]
    delta = s_pct - o_pct
    print(f"{label:<10} {count:>12,} {s_pct:>11.2f}% {o_pct:>11.2f}% {delta:>+7.3f}%")

print("-" * 60)
print(f"{'Total':<10} {df_sample.shape[0]:>12,}")
print("\n✅ Stratification preserved! Class proportions maintained.")


🔍 Sample Class Distribution (Integrity Check):
Class             Count     Sample %   Original %        Δ
------------------------------------------------------------
0               165,083       82.54%       82.54%  -0.000%
1                34,917       17.46%       17.46%  +0.000%
------------------------------------------------------------
Total           200,000

✅ Stratification preserved! Class proportions maintained.


In [6]:
# Free up memory from original dataset
del df, X, y
import gc
gc.collect()
print("🧹 Memory cleaned: Original dataset removed from memory.")

🧹 Memory cleaned: Original dataset removed from memory.


In [7]:
from pycaret.classification import (
    setup, compare_models, plot_model, save_model, create_model, tune_model
)

print("🚀 Initializing PyCaret Classification Environment...")
print("   GPU Mode: DISABLED (Intel OpenCL Issue)")
print("   Normalization: ENABLED")
print("   Session ID: 42 (Reproducibility)")
print("=" * 60)

🚀 Initializing PyCaret Classification Environment...
   GPU Mode: DISABLED (Intel OpenCL Issue)
   Normalization: ENABLED
   Session ID: 42 (Reproducibility)


In [8]:
# Initialize PyCaret setup
clf_setup = setup(
    data=df_sample,
    target=TARGET_COLUMN,
    session_id=RANDOM_STATE,
    n_jobs=-1,              # Use all CPU cores
    use_gpu=False,          # GPU disabled to prevent LightGBM/Intel OpenCL crash
    normalize=True,         # Apply scaling
    log_experiment=False,   # Lightweight - no MLflow logging
    verbose=True,           # Show setup information
    html=False              # Disable HTML output for cleaner logs
)

                    Description             Value
0                    Session id                42
1                        Target             Label
2                   Target type            Binary
3           Original data shape      (200000, 21)
4        Transformed data shape      (200000, 21)
5   Transformed train set shape      (140000, 21)
6    Transformed test set shape       (60000, 21)
7              Numeric features                20
8                    Preprocess              True
9               Imputation type            simple
10           Numeric imputation              mean
11       Categorical imputation              mode
12                    Normalize              True
13             Normalize method            zscore
14               Fold Generator   StratifiedKFold
15                  Fold Number                10
16                     CPU Jobs                -1
17                      Use GPU             False
18               Log Experiment             False


In [9]:
# Define models to compare
BATTLE_MODELS = [
    'lightgbm',   # LightGBM - Fast gradient boosting
    'xgboost',    # XGBoost - Extreme gradient boosting
    'catboost',   # CatBoost - Categorical boosting
    'rf',         # Random Forest
    'et',         # Extra Trees
    'dt',         # Decision Tree
    'ada',        # AdaBoost
    'gbc'         # Gradient Boosting Classifier
]

print("⚔️  THE BATTLE BEGINS!")
print("=" * 60)
print("Contenders:")
for i, model in enumerate(BATTLE_MODELS, 1):
    print(f"  {i}. {model.upper()}")
print("\n📏 Metrics: 5-Fold CV | Optimizing for F1 Score")
print("=" * 60)

⚔️  THE BATTLE BEGINS!
Contenders:
  1. LIGHTGBM
  2. XGBOOST
  3. CATBOOST
  4. RF
  5. ET
  6. DT
  7. ADA
  8. GBC

📏 Metrics: 5-Fold CV | Optimizing for F1 Score


In [10]:
# Run the comparison
print("\n🏁 Starting model comparison (this may take several minutes)...\n")

best_model = compare_models(
    include=BATTLE_MODELS,
    fold=5,                 # 5-fold CV for speed
    sort='F1',              # Optimize for F1 Score
    n_select=1,             # Return only the best model
    verbose=True            # Show progress
)

print("\n" + "=" * 60)
print("🏆 CHAMPION SELECTED!")
print(f"   Best Model: {type(best_model).__name__}")
print("=" * 60)


🏁 Starting model comparison (this may take several minutes)...



                                    Model  Accuracy     AUC  Recall   Prec.  \
et                 Extra Trees Classifier    0.9978  0.9989  0.9932  0.9943   
xgboost         Extreme Gradient Boosting    0.9971  0.9999  0.9885  0.9951   
lightgbm  Light Gradient Boosting Machine    0.9970  0.9998  0.9871  0.9959   
rf               Random Forest Classifier    0.9970  0.9995  0.9883  0.9946   
dt               Decision Tree Classifier    0.9970  0.9948  0.9914  0.9914   
catboost              CatBoost Classifier    0.9966  0.9998  0.9854  0.9951   
gbc          Gradient Boosting Classifier    0.9944  0.9988  0.9749  0.9928   
ada                  Ada Boost Classifier    0.9845  0.9975  0.9307  0.9793   

              F1   Kappa     MCC  TT (Sec)  
et        0.9938  0.9925  0.9925     1.180  
xgboost   0.9918  0.9900  0.9900     1.158  
lightgbm  0.9915  0.9897  0.9897     1.130  
rf        0.9914  0.9896  0.9896     3.096  
dt        0.9914  0.9896  0.9896     0.494  
catboost  0.9902  

In [11]:
import matplotlib.pyplot as plt
import shutil

print("📊 Generating visualizations...")
print(f"   Output directory: {RESULTS_DIR}")
print("=" * 60)

📊 Generating visualizations...
   Output directory: D:\Projects\networkdetection\networkdetection\experiments\results


In [12]:
# Helper function to save PyCaret plots
def save_pycaret_plot(model, plot_type, filename):
    """
    Generate and save PyCaret plot.
    
    Args:
        model: Trained PyCaret model
        plot_type: Type of plot (e.g., 'confusion_matrix', 'auc', 'feature')
        filename: Output filename (without extension)
    """
    try:
        # PyCaret's plot_model saves to current directory by default
        plot_model(model, plot=plot_type, save=True)
        
        # Default name pattern used by PyCaret
        default_name = f"{plot_type.replace('_', ' ').title()}.png"
        
        # Alternative patterns PyCaret might use
        possible_names = [
            default_name,
            f"{plot_type}.png",
            f"{plot_type.title()}.png",
            "Confusion Matrix.png",
            "Feature Importance.png",
            "AUC.png"
        ]
        
        # Find and move the generated file
        for name in possible_names:
            if os.path.exists(name):
                target_path = RESULTS_DIR / filename
                shutil.move(name, target_path)
                print(f"  ✅ Saved: {filename}")
                return target_path
        
        print(f"  ⚠️ Could not locate generated plot for {plot_type}")
        return None
        
    except Exception as e:
        print(f"  ❌ Error generating {plot_type}: {str(e)}")
        return None

In [13]:
# Generate and save Confusion Matrix
print("\n Generating Confusion Matrix...")
save_pycaret_plot(best_model, 'confusion_matrix', 'confusion_matrix.png')


 Generating Confusion Matrix...
  ✅ Saved: confusion_matrix.png


WindowsPath('D:/Projects/networkdetection/networkdetection/experiments/results/confusion_matrix.png')

In [14]:
# Generate and save Feature Importance
print("\n Generating Feature Importance...")
save_pycaret_plot(best_model, 'feature', 'feature_importance.png')


 Generating Feature Importance...
  ✅ Saved: feature_importance.png


WindowsPath('D:/Projects/networkdetection/networkdetection/experiments/results/feature_importance.png')

In [15]:
# Generate and save AUC/ROC Curve
print("\n Generating ROC/AUC Curve...")
save_pycaret_plot(best_model, 'auc', 'auc.png')


 Generating ROC/AUC Curve...
  ✅ Saved: auc.png


WindowsPath('D:/Projects/networkdetection/networkdetection/experiments/results/auc.png')

In [16]:
# Display generated visualizations
print("\n Generated Artifacts:")
print("=" * 60)
for f in RESULTS_DIR.glob('*.png'):
    print(f"  📊 {f.name}")


 Generated Artifacts:
  📊 auc.png
  📊 confusion_matrix.png
  📊 feature_importance.png


In [17]:
# Save the champion model
model_path = MODELS_DIR / 'pycaret_champion'

print("\n Saving champion model...")
save_model(best_model, str(model_path))
print(f"\n Model saved to: {model_path}")
print(f"   File: pycaret_champion.pkl")


 Saving champion model...
Transformation Pipeline and Model Successfully Saved

 Model saved to: D:\Projects\networkdetection\networkdetection\models\pycaret_champion
   File: pycaret_champion.pkl


In [18]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           BENCHMARK COMPLETE - SUMMARY                     ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║   Data Pipeline:                                           ║
║     ├─ Source: data/processed_randomforest/train.csv         ║
║     ├─ Original: ~1.3M+ samples                              ║
║     └─ Stratified Sample: 200,000 samples                    ║
║                                                              ║
║    Battle Results:                                          ║
║     ├─ Algorithms Tested: 8                                  ║
║     ├─ Cross-Validation: 5-Fold                              ║
║     ├─ Primary Metric: F1 Score                              ║
║     └─ Champion: {champion_name:<35} ║
║                                                              ║
║   Artifacts Generated:                                     ║
║     ├─ experiments/results/confusion_matrix.png              ║
║     ├─ experiments/results/feature_importance.png            ║
║     ├─ experiments/results/auc.png                           ║
║     └─ models/pycaret_champion.pkl                           ║
║                                                              ║
╠══════════════════════════════════════════════════════════════╣
║   Next Steps:                                               ║
║     1. Review metrics from compare_models() output           ║
║     2. Fine-tune hyperparameters with tune_model()           ║
║     3. Train on full dataset with identified architecture    ║
║     4. Evaluate on held-out test set                         ║
╚══════════════════════════════════════════════════════════════╝
""".format(champion_name=type(best_model).__name__))


╔══════════════════════════════════════════════════════════════╗
║           BENCHMARK COMPLETE - SUMMARY                     ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║   Data Pipeline:                                           ║
║     ├─ Source: data/processed_randomforest/train.csv         ║
║     ├─ Original: ~1.3M+ samples                              ║
║     └─ Stratified Sample: 200,000 samples                    ║
║                                                              ║
║    Battle Results:                                          ║
║     ├─ Algorithms Tested: 8                                  ║
║     ├─ Cross-Validation: 5-Fold                              ║
║     ├─ Primary Metric: F1 Score                              ║
║     └─ Champion: ExtraTreesClassifier                ║
║                                                              ║
║   Artifacts Generated:             

In [19]:
# Display model info
print("\n Champion Model Details:")
print("=" * 60)
print(best_model)


 Champion Model Details:
ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)
